In [1]:
#!pip install pandas-gbq

In [1]:
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
import requests
import datetime
import re
import json
from datetime import date, timedelta
#client = bigquery.Client(location="us-central1")
client = bigquery.Client(location="US")
#print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, expr,first, last,when, split, col,lit, concat, date_format,to_utc_timestamp,from_utc_timestamp,to_timestamp, regexp_replace
from pyspark.sql.functions import sum as _sum


In [2]:
spark = SparkSession.builder.appName("StorageAllocation").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/20 16:25:25 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/04/20 16:25:25 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/04/20 16:25:25 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/04/20 16:25:25 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/04/20 16:25:32 ERROR org.apache.spark.SparkContext: Error initializing SparkContext.
java.io.FileNotFoundException: File not found: gs://dataproc-temp-us-central1-83748865223-voxjkqak/ba48077b-3fdc-4459-8e8c-b9fc26207e36/spark-job-history
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.getFileStatus(GoogleHadoopFileSystemBase.java:961)
	at org.apache.spark.deploy.history.EventLogFileWriter.requireLogBaseDirAsDirectory(EventLogFileWriters.scala:77)
	at org.apache.spark.deploy.history.SingleEventLogFileWriter.star

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.io.FileNotFoundException: File not found: gs://dataproc-temp-us-central1-83748865223-voxjkqak/ba48077b-3fdc-4459-8e8c-b9fc26207e36/spark-job-history
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.getFileStatus(GoogleHadoopFileSystemBase.java:961)
	at org.apache.spark.deploy.history.EventLogFileWriter.requireLogBaseDirAsDirectory(EventLogFileWriters.scala:77)
	at org.apache.spark.deploy.history.SingleEventLogFileWriter.start(EventLogFileWriters.scala:221)
	at org.apache.spark.scheduler.EventLoggingListener.start(EventLoggingListener.scala:83)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:612)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.D

In [9]:
current_date = datetime.datetime.today()

dias_atras = 3
#dias_atras = int(dias_atras)

str_day_process = (current_date - timedelta(days = dias_atras)).strftime("%Y-%m-%d")
str_day_process_dt = pd.to_datetime(str_day_process)
str_lastday = (str_day_process_dt - timedelta(days = 1)).strftime("%Y-%m-%d")
str_today = (str_day_process_dt + timedelta(days = 1)).strftime("%Y-%m-%d")

#STR_YEARMONTH = (current_date - timedelta(days = dias_atras)).strftime("%Y%m")
#str_day_process = "2022-07-25"
#STR_YEARMONTH = "201908"
print("Dia Analizado --> ",str_day_process," ; ","extraido de :",str_lastday," - ",str_today)

Dia Analizado -->  2023-04-15  ;  extraido de : 2023-04-14  -  2023-04-16


## YT1 

In [33]:
environment = 'YT1'

### Read YT1 Mongo DB Collection

In [35]:
df_mongo_yt1 = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','raw.yt1_mongodb_collection')\
.option("filter","date between '%s 00:00:00' and '%s 23:59:59' "%(str_day_process,str_day_process))\
.load()

df_mongo_yt1 = df_mongo_yt1.withColumn("environment",lit(environment))
df_mongo_yt1 = df_mongo_yt1.withColumn('date_normalized', to_date(col("date")))
df_mongo_agg = df_mongo_yt1.groupBy(['date_normalized','tenant','real_name','environment'])\
                        .agg(_sum('size_db').alias('size_db'), \
                        _sum('size_storage').alias('size_storage'), \
                        _sum('size_index').alias('size_index'))

### YT1 Storage Cost Extraction - GCP

In [12]:
environment = 'YT1'

In [13]:
project_number = '260718309650'

In [14]:
GCP_cost_table = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D')\
.option("filter","JSON_VALUE(project.number) = '%s' and usage_start_time between '%s 00:00:00' and '%s 23:59:59' "%(project_number,str_lastday, str_today))\
.load()

In [15]:
GCP_cost_table = GCP_cost_table.select(col("service.description").alias('service_description'),
                                          col("sku.description").alias('sku_description'),
                                          col("usage_start_time"),
                                          col("project.id").alias('id_project'),
                                          col("cost"),
                                          col("usage.amount").alias('usage_amount'),
                                          col("usage.unit").alias('usage_unit'),
                                          col("credits")[0].alias('credits_1'),
                                          col("credits")[1].alias('credits_2'),
                                          col("cost_type")
                                         )   

GCP_cost_table = GCP_cost_table.select(col('service_description'),
                                          col('sku_description'),
                                          col("usage_start_time"),
                                          col('id_project'),
                                          col("cost"),
                                          col('usage_amount'),
                                          col('usage_unit'),
                                          col("credits_1.name").alias('credits_1_name'),
                                          col("credits_1.amount").alias('credits_1_amount'),
                                          col("credits_1.type").alias('credits_1_type'),
                                          col("credits_2.name").alias('credits_2_name'),
                                          col("credits_2.amount").alias('credits_2_amount'),
                                          col("credits_2.type").alias('credits_2_type'),
                                          col("cost_type")
                                         ) 
#GCP_cost_table.show()

In [16]:
GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%DISCOUNT%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%DISCOUNT%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))

GCP_cost_table = GCP_cost_table.withColumn("credits_promotion",when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%PROMOTION%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%PROMOTION%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["credits_discounts",'credits_promotion'])

In [17]:
GCP_cost_table = GCP_cost_table.withColumn("storage_cost",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn("storage_discount",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['credits_discounts']))
GCP_cost_table = GCP_cost_table.withColumn('total_storage_after_discount', GCP_cost_table['storage_cost'] + GCP_cost_table['credits_discounts'])


In [18]:
GCP_cost_table = GCP_cost_table.filter((GCP_cost_table['sku_description'].like('%SSD%')))

In [19]:
GCP_cost_table_agg = GCP_cost_table.groupBy(['usage_start_time'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'), \
                        _sum('credits_promotion').alias('daily_promotion_credits'))

df_cost_storage_gcp = GCP_cost_table.groupBy(['usage_start_time','sku_description'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('storage_discount').alias('daily_storage_discount_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'))

df_cost_storage_gcp = df_cost_storage_gcp.withColumn('date_normalized', to_date(from_utc_timestamp(col("usage_start_time"),"America/Los_Angeles")))

df_cost_storage_gcp_agg = df_cost_storage_gcp.filter((df_cost_storage_gcp['date_normalized']==str_day_process)).groupBy(['date_normalized'])\
                        .agg(_sum('daily_storage_cost_sum').alias('daily_storage_cost_sum'), \
                        _sum('daily_storage_discount_sum').alias('daily_storage_discount_sum'), \
                        _sum('daily_storage_after_discount_sum').alias('daily_storage_after_discount_sum'))


### YT1 JOIN Storage Usage with GCP Costs

In [ ]:
df_results = df_mongo_agg.join(df_cost_storage_gcp_agg,['date_normalized'],"left")
df_results = df_results.join(df_mongo_agg.groupBy('date_normalized').agg(_sum('size_db').alias('size_db_tot_day'), \
                        _sum('size_storage').alias('size_storage_tot_day'), \
                        _sum('size_index').alias('size_index_tot_day'))\
                             ,['date_normalized'],"left")


df_results = df_results.withColumn("storage_cost_assigned",((df_results['daily_storage_cost_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("storage_cost_assigned_after_discounts",((df_results['daily_storage_after_discount_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("size_storage_percentage",((df_results['size_storage']/df_results['size_storage_tot_day'])*100))


df_results_upload = df_results[['environment','date_normalized','tenant','real_name',
                                'size_storage','size_storage_percentage',
                                'storage_cost_assigned','storage_cost_assigned_after_discounts']]


bucket = "finops-outputs"
spark.conf.set('temporaryGcsBucket', bucket)
print("Write data in Big Query Tables")

df_results_upload.write.format('bigquery') \
    .option('table', 'saas-analytics-io.processed.storage_cost_assign_v2') \
    .mode('append') \
    .save()

## VT1 

In [50]:
environment = 'VT1'

### Read VT1 Mongo DB Collection

In [51]:
df_mongo_vt1 = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','raw.vt1_mongodb_collection')\
.option("filter","date between '%s 00:00:00' and '%s 23:59:59' "%(str_day_process,str_day_process))\
.load()

df_mongo_vt1 = df_mongo_vt1.withColumn("environment",lit(environment))
df_mongo_vt1 = df_mongo_vt1.withColumn('date_normalized', to_date(col("date")))
df_mongo_agg = df_mongo_vt1.groupBy(['date_normalized','tenant','real_name','environment'])\
                        .agg(_sum('size_db').alias('size_db'), \
                        _sum('size_storage').alias('size_storage'), \
                        _sum('size_index').alias('size_index'))

### VT1 Storage Cost Extraction - GCP

In [52]:
environment = 'YT1'

In [53]:
project_number = '884190693971'

In [54]:
GCP_cost_table = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D')\
.option("filter","JSON_VALUE(project.number) = '%s' and usage_start_time between '%s 00:00:00' and '%s 23:59:59' "%(project_number,str_lastday, str_today))\
.load()

In [55]:
GCP_cost_table = GCP_cost_table.select(col("service.description").alias('service_description'),
                                          col("sku.description").alias('sku_description'),
                                          col("usage_start_time"),
                                          col("project.id").alias('id_project'),
                                          col("cost"),
                                          col("usage.amount").alias('usage_amount'),
                                          col("usage.unit").alias('usage_unit'),
                                          col("credits")[0].alias('credits_1'),
                                          col("credits")[1].alias('credits_2'),
                                          col("cost_type")
                                         )   

GCP_cost_table = GCP_cost_table.select(col('service_description'),
                                          col('sku_description'),
                                          col("usage_start_time"),
                                          col('id_project'),
                                          col("cost"),
                                          col('usage_amount'),
                                          col('usage_unit'),
                                          col("credits_1.name").alias('credits_1_name'),
                                          col("credits_1.amount").alias('credits_1_amount'),
                                          col("credits_1.type").alias('credits_1_type'),
                                          col("credits_2.name").alias('credits_2_name'),
                                          col("credits_2.amount").alias('credits_2_amount'),
                                          col("credits_2.type").alias('credits_2_type'),
                                          col("cost_type")
                                         ) 
#GCP_cost_table.show()

In [56]:
GCP_cost_table = GCP_cost_table.withColumn("credits_discounts",when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%DISCOUNT%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%DISCOUNT%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%DISCOUNT%'))&
                                                (GCP_cost_table['credits_2_type'].like('%DISCOUNT%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))

GCP_cost_table = GCP_cost_table.withColumn("credits_promotion",when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                                    ((~(GCP_cost_table['credits_2_type'].like('%PROMOTION%')))|
                                                                     (GCP_cost_table['credits_2_type'].isNull())),GCP_cost_table['credits_1_amount'])
                                          .when(((~(GCP_cost_table['credits_1_type'].like('%PROMOTION%')))
                                                |(GCP_cost_table['credits_1_type'].isNull()))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_2_amount'])
                                          .when((GCP_cost_table['credits_1_type'].like('%PROMOTION%'))&
                                                (GCP_cost_table['credits_2_type'].like('%PROMOTION%')),GCP_cost_table['credits_1_amount']+GCP_cost_table['credits_2_amount']))
GCP_cost_table = GCP_cost_table.na.fill(value=0,subset=["credits_discounts",'credits_promotion'])

In [57]:
GCP_cost_table = GCP_cost_table.withColumn("storage_cost",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['cost']))
GCP_cost_table = GCP_cost_table.withColumn("storage_discount",when((GCP_cost_table['sku_description'].like('%SSD%')),GCP_cost_table['credits_discounts']))
GCP_cost_table = GCP_cost_table.withColumn('total_storage_after_discount', GCP_cost_table['storage_cost'] + GCP_cost_table['credits_discounts'])


In [58]:
GCP_cost_table = GCP_cost_table.filter((GCP_cost_table['sku_description'].like('%SSD%')))

In [59]:
GCP_cost_table_agg = GCP_cost_table.groupBy(['usage_start_time'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'), \
                        _sum('credits_promotion').alias('daily_promotion_credits'))

df_cost_storage_gcp = GCP_cost_table.groupBy(['usage_start_time','sku_description'])\
                        .agg(_sum('storage_cost').alias('daily_storage_cost_sum'), \
                        _sum('storage_discount').alias('daily_storage_discount_sum'), \
                        _sum('total_storage_after_discount').alias('daily_storage_after_discount_sum'))

df_cost_storage_gcp = df_cost_storage_gcp.withColumn('date_normalized', to_date(from_utc_timestamp(col("usage_start_time"),"America/Los_Angeles")))

df_cost_storage_gcp_agg = df_cost_storage_gcp.filter((df_cost_storage_gcp['date_normalized']==str_day_process)).groupBy(['date_normalized'])\
                        .agg(_sum('daily_storage_cost_sum').alias('daily_storage_cost_sum'), \
                        _sum('daily_storage_discount_sum').alias('daily_storage_discount_sum'), \
                        _sum('daily_storage_after_discount_sum').alias('daily_storage_after_discount_sum'))


### VT1 JOIN Storage Usage with GCP Costs

In [60]:
df_results = df_mongo_agg.join(df_cost_storage_gcp_agg,['date_normalized'],"left")
df_results = df_results.join(df_mongo_agg.groupBy('date_normalized').agg(_sum('size_db').alias('size_db_tot_day'), \
                        _sum('size_storage').alias('size_storage_tot_day'), \
                        _sum('size_index').alias('size_index_tot_day'))\
                             ,['date_normalized'],"left")


df_results = df_results.withColumn("storage_cost_assigned",((df_results['daily_storage_cost_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("storage_cost_assigned_after_discounts",((df_results['daily_storage_after_discount_sum']/df_results['size_storage_tot_day'])*df_results['size_storage']))
df_results = df_results.withColumn("size_storage_percentage",((df_results['size_storage']/df_results['size_storage_tot_day'])*100))


df_results_upload = df_results[['environment','date_normalized','tenant','real_name',
                                'size_storage','size_storage_percentage',
                                'storage_cost_assigned','storage_cost_assigned_after_discounts']]


#bucket = "finops-outputs"
#spark.conf.set('temporaryGcsBucket', bucket)
#print("Write data in Big Query Tables")
#
#df_results_upload.write.format('bigquery') \
#    .option('table', 'saas-analytics-io.processed.storage_cost_assign_v2') \
#    .mode('append') \
#    .save()

In [62]:
df_results_upload.show()

+-----------+---------------+----------+--------------------+------------+-----------------------+---------------------+-------------------------------------+
|environment|date_normalized|    tenant|           real_name|size_storage|size_storage_percentage|storage_cost_assigned|storage_cost_assigned_after_discounts|
+-----------+---------------+----------+--------------------+------------+-----------------------+---------------------+-------------------------------------+
|        VT1|     2023-04-15|     FEDEX|          vizixEvent|       0.003|   1.659573832503639E-4|  3.07518350737653E-5|                  3.07518350737653E-5|
|        VT1|     2023-04-15|      IKEA|STARFLEX_STATUS_C...|         0.0|                    0.0|                  0.0|                                  0.0|
|        VT1|     2023-04-15|      LUER|      thingSequences|         0.0|                    0.0|                  0.0|                                  0.0|
|        VT1|     2023-04-15|     FEDEX|      